# Introduction to Data Science
## Homework 2

Student Name: Benjamin Jakubowski (buj201)
***

### Part 1: Case study
- Read [this article](http://www.nytimes.com/2012/02/19/magazine/shopping-habits.html) in the New York Times
- Use what we've learned in class as well as from the book and our readings to describe how one could set this up as a predictive modeling problem, such that they could have gotten the results that they did

Target was trying to predict which women shoppers were pregnant based on their purchases. This could be constructed as one (or more) of several supervised learning problems, including: (1) a classification problem (classify women as pregnant/non-pregnant) and (2) a regression problem (predict time until expected delivery). To connect back to the reading, the classification problem could be approached using a decision tree (as presented in Chapter 3 of the reading), though it is unlikely Target used such a naive approach.

Note that regardless of the specific formulation of the prediction problem, this is a supervised learning problem thanks to the fact Target has a baby-shower registry. Pregnant women willingly and freely give Target information about their expected delivery dates. By linking this baby-shower registry with purchasing records, Target was able to identify predictors of pregnancy (i.e. products women purchase in greater amounts during pregnancy). Ultimately, Target was able to build models using these predictors that allowed it to accurately predict which women were pregnant and when they were due. This allowed Target's marketing team to target these women with the hopes of making Target their supplier of baby (and non-baby) related items for years to come.

### Part 2: Exploring data in the command line
For this part we will be using the data file located in `"data/advertising_events.csv"`. This file consists of records that pertain to some advertising events on a given day. There are 4 comma separated columns in this order: `userid`, `timestamp`, `domain`, and `action`. These fields are of type `int`, `int`, `string`, and `int` respectively. Answer the following questions using Linux/Unix bash commands. All questions can be answered in one line (sometimes, with pipes)! Some questions will have many possible solutions. Don't forget that in IPython notebooks you must prefix all bash commands with an exclamation point, i.e. `"!command arguments"`.

1\. How many records are in this file?

In [41]:
!cat data/advertising_events.csv | wc -l

   10341


2\. How many unique users are in this file?

In [42]:
!cut -d',' -f1 ./data/advertising_events.csv | sort -n | uniq | wc -l

     732


3\. Rank all domains by the number of visits they received in descending order.

In [43]:
!cut -d',' -f3 ./data/advertising_events.csv | sort | uniq -c | sort -n -r

3114 google.com
2092 facebook.com
1036 youtube.com
1034 yahoo.com
1022 baidu.com
 513 wikipedia.org
 511 amazon.com
 382 qq.com
 321 twitter.com
 316 taobao.com


4\. List all records for the user with user id 37.

In [44]:
!grep ^37, ./data/advertising_events.csv

37,648061658,google.com,0
37,642479972,google.com,2
37,644493341,facebook.com,2
37,654941318,facebook.com,1
37,649979874,baidu.com,1
37,653061949,yahoo.com,1
37,655020469,google.com,3
37,640878012,amazon.com,0
37,659864136,youtube.com,1
37,640361378,yahoo.com,1
37,653862134,facebook.com,0
37,648828970,youtube.com,0


### Part 3: Dealing with messy data
Not all data you will deal with is going to be clean. In fact, much of it will be very messy! For example, we have the HTML page that lists the contributors to Facebook's [osquery](https://github.com/facebook/osquery) project that is hosted on [Github.com](https://github.com). In this case, all we are interested in are the contributors and how many commits each of them has. Given the HTML page in `"data/osquery_contributors.html"` you will sift through tons of irrelevant data so that you can build a useful data structure.

Notice that the first six (out of 59 total) contributors are named "theopolis", "marpaia", "javuto", "jedi22", "unixist", and "mofarrell". They have 553, 477, 104, 49, 30, 25 commits respectively.

![Screenshot](./data/osquery_contributors.png)

To get a better of understanding of how this data is stored in the file, try searching through the raw data file for these usernames to look for any patterns. Your final dictionary should have 59 elements!

1\. Turn this data into a Python dictionary called `contributors` where the keys are the contributor names and the values are the number of commits that each contributor has.

In [45]:
import re # you might find this package useful

contributors = dict()

html_code = open('./data/osquery_contributors.html', "r").readlines()

for line in html_code:
    if re.search('commits\?author', line):
        name_count_html_frag_large = re.split('author=', line)[1]
        name_count_html_frag_small = re.split(' ', name_count_html_frag_large)[0]
        name = re.search(r'^[a-zA-Z0-9]*', name_count_html_frag_small).group()
        number_contributions = re.search(r'[0-9]*$',name_count_html_frag_small).group()
        contributors[name] = number_contributions
        
print contributors

{'shawndavenport': '1', 'jedi22': '49', 'schettino72': '2', 'jamesgpearce': '2', 'marpaia': '477', 'wxsBSD': '20', 'blakefrantz': '6', 'yetanotherhacker': '1', 'lwhsu': '22', 'mimeframe': '3', 'd0ugal': '1', 'kost': '1', 'polachok': '14', 'achmiel': '3', 'vmauge': '8', 'theopolis': '553', 'yannick': '1', 'mark': '1', 'SimplyAhmazing': '1', 'timzimmermann': '2', 'mgoffin': '2', 'deniszh': '3', 'Anubisss': '2', 'vlajos': '1', 'dreid': '1', 'astanway': '6', 'arubdesu': '1', 'sharvilshah': '23', 'jreese': '2', 'mtmcgrew': '1', 'justintime32': '1', 'nlsun': '3', 'mathieuk': '2', 'ecin': '1', 'blackfist': '1', 'apage43': '1', 'zwass': '14', 'mofarrell': '25', 'maclennann': '6', 'quad': '1', 'arirubinstein': '4', 'brandt': '3', 'rjeczalik': '1', 'ga2arch': '2', 'maus': '9', 'alex': '1', 'unixist': '30', 'cdown': '4', 'javuto': '104', 'larzconwell': '1', 'castrapel': '2', 'jacknagz': '1', 'akshaydixi': '5', 'mlw': '2', 'stevenhilder': '1', 'glensc': '2', 'tburgin': '1', 'DavidGosselin': '1', '

### Part 4: Dealing with data Pythonically

In [46]:
# You might find these packages useful. You may import any other you want!
import pandas as pd
import numpy as np

1\. Download the data set `"data/ads_dataset.tsv"` and load it into a Python Pandas data frame called `ads`.

In [47]:
ads = pd.read_csv('./data/ads_dataset.tsv', delim_whitespace=True)

2\. Write a Python function called `getDfSummary()` that does the following:
- Takes as input a data frame
- For each variable in the data frame calculates the following features:
  - `number_nan` to count the number of missing not-a-number values
  - Ignoring missing, NA, and Null values:
    - `number_distinct` to count the number of distinct values a variable can take on
    - `mean`, `max`, `min`, `std` (standard deviation), and `25%`, `50%`, `75%` to correspond to the appropriate percentiles
- All of these new features should be loaded in a new data frame. Each row of the data frame should be a variable from the input data frame, and the columns should be the new summary features.
- Returns this new data frame containing all of the summary information

Hint: The pandas `describe()` method returns a useful series of values that can be used here.

In [48]:
def getDfSummary(input_data):
    
    #Use pd.describe() to obtain mean, max, min, std, quantiles, and count for each variable
    input_data_summary = input_data.describe().transpose()
    
    #remove count from summary table
    input_data_summary = input_data_summary.drop('count', 1)
    
    #add column for number_nan
    input_data_summary['number_nan'] = pd.Series(input_data.isnull().sum(), index=input_data_summary.index)

    #count unique values for each variable and add as column in dataframe
    unique_counts = {}
    for variable in input_data.columns.values:
        unique_counts[variable] = len(input_data[variable].value_counts())
        input_data_summary['number_distinct'] = pd.Series(unique_counts, index=unique_counts.keys())
    
    return input_data_summary

3\. How long does it take for your `getDfSummary()` function to work on your `ads` data frame? Show us the results below.

Hint: `%timeit getDfSummary(ads)`

In [49]:
%timeit getDfSummary(ads)

getDfSummary(ads)

1 loops, best of 3: 321 ms per loop


,mean,std,min,25%,50%,75%,max,number_nan,number_distinct
isbuyer,0.042632,0.202027,0.0000,0,0,0.000000,1.00000,0,2
buy_freq,1.240653,0.782228,1.0000,1,1,1.000000,15.00000,52257,10
visit_freq,1.852777,2.921820,0.0000,1,1,2.000000,84.00000,0,64
buy_interval,0.210008,3.922016,0.0000,0,0,0.000000,174.62500,0,295
sv_interval,5.825610,17.595442,0.0000,0,0,0.104167,184.91670,0,5886
expected_time_buy,-0.198040,4.997792,-181.9238,0,0,0.000000,84.28571,0,348
expected_time_visit,-10.210786,31.879722,-187.6156,0,0,0.000000,91.40192,0,15135
last_buy,64.729335,53.476658,0.0000,18,51,105.000000,188.00000,0,189
last_visit,64.729335,53.476658,0.0000,18,51,105.000000,188.00000,0,189
multiple_buy,0.006357,0.079479,0.0000,0,0,0.000000,1.00000,0,2


4\. Using the results returned from `getDfSummary()`, which fields, if any, contain missing `NaN` values?

The only field that contains missing NaN values is buy_freq (with 52257 missing values).

5\. For the fields with missing values, does it look like the data is missing at random? Are there any other fields that correlate perfectly, or predict that the data is missing? If missing, what should the data value be?

Hint: create another data frame that has just the records with a missing value. Get a summary of this data frame using `getDfSummary()` and compare the differences. Do some feature distributions change dramatically?

In [50]:
Nan_data = ads.loc[pd.isnull(ads['buy_freq'])]

getDfSummary(Nan_data)

,mean,std,min,25%,50%,75%,max,number_nan,number_distinct
isbuyer,0.000000,0.000000,0.0000,0,0,0.000000,0.00000,0,1
buy_freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52257,0
visit_freq,1.651549,2.147955,1.0000,1,1,2.000000,84.00000,0,48
buy_interval,0.000000,0.000000,0.0000,0,0,0.000000,0.00000,0,1
sv_interval,5.686388,17.623555,0.0000,0,0,0.041667,184.91670,0,5112
expected_time_buy,0.000000,0.000000,0.0000,0,0,0.000000,0.00000,0,1
expected_time_visit,-9.669298,31.239030,-187.6156,0,0,0.000000,91.40192,0,13351
last_buy,65.741317,53.484622,0.0000,19,52,106.000000,188.00000,0,189
last_visit,65.741317,53.484622,0.0000,19,52,106.000000,188.00000,0,189
multiple_buy,0.000000,0.000000,0.0000,0,0,0.000000,0.00000,0,1


The data missing in buy_freq is definitely not missing at random. This is obvious from the summary data frame printed above. When we look at only those records with missing buy_freq values, isbuyer, buy_interval, expected_time_buy, and multiple_buy all become constant: if buy_freq is NaN, these variables always take on the value 0.  This makes intuitive sense- if a record is missing buy_freq, it is classified as a non-buyer and these related variables take on 0 values. Based on this inference, if we wanted to impute the missing value of buy_freq we would be justified in assigning it the value 0.

6\. Which variables are binary?

There are four binary variables- isbuyer, mutiple_by, multiple_visit, and y_buy.

7\. Generate a correlation matrix for the `ads` data frame. Is there any redundancy in the data? Are there any features that aren't needed?

In [51]:
ads.corr()

,isbuyer,buy_freq,visit_freq,buy_interval,sv_interval,expected_time_buy,expected_time_visit,last_buy,last_visit,multiple_buy,multiple_visit,uniq_urls,num_checkins,y_buy
isbuyer,1.000000,NaN,0.326373,0.253749,0.037496,-0.187782,-0.080492,-0.089678,-0.089678,0.379045,0.231174,-0.006641,-0.004424,0.112434
buy_freq,NaN,1.000000,0.487548,0.398839,0.000280,-0.291767,0.060817,-0.126793,-0.126793,0.735054,0.154837,0.042624,0.042764,0.128118
visit_freq,0.326373,0.487548,1.000000,0.262660,0.140983,-0.147219,-0.121919,-0.171388,-0.171388,0.354157,0.472163,0.039079,0.048881,0.118092
buy_interval,0.253749,0.398839,0.262660,1.000000,0.042134,0.005762,0.010266,-0.046015,-0.046015,0.669442,0.083427,0.013895,0.008804,0.068453
sv_interval,0.037496,0.000280,0.140983,0.042134,1.000000,0.001817,0.017179,-0.150368,-0.150368,0.022634,0.534310,0.094210,0.073020,0.011096
expected_time_buy,-0.187782,-0.291767,-0.147219,0.005762,0.001817,1.000000,0.053677,-0.001402,-0.001402,-0.495407,-0.059168,-0.008078,-0.005148,-0.038141
expected_time_visit,-0.080492,0.060817,-0.121919,0.010266,0.017179,0.053677,1.000000,-0.284973,-0.284973,-0.013129,-0.516888,-0.083879,-0.051314,0.011826
last_buy,-0.089678,-0.126793,-0.171388,-0.046015,-0.150368,-0.001402,-0.284973,1.000000,1.000000,-0.058602,-0.183995,0.268582,0.142725,-0.059171
last_visit,-0.089678,-0.126793,-0.171388,-0.046015,-0.150368,-0.001402,-0.284973,1.000000,1.000000,-0.058602,-0.183995,0.268582,0.142725,-0.059171
multiple_buy,0.379045,0.735054,0.354157,0.669442,0.022634,-0.495407,-0.013129,-0.058602,-0.058602,1.000000,0.123419,0.004246,0.005247,0.109926


The maximum pairwise correlation in the dataframe is between multiple_buy and buy_frequency, with a correlation of 0.73504. Since multiple_buy is also highly correlated with buy_interval, it may be redundant and not contain additional information about the customer.